In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import os
os.chdir('/content/drive/MyDrive/aghil')
os.listdir()


['data', 'csv_files', 'outputs', 'notebook_1.ipynb', 'ds_report.pdf']

In [4]:
os.listdir('data')


['fear_greed_index.csv', 'historical_data.csv']

In [5]:
import pandas as pd

trades = pd.read_csv('data/historical_data.csv')
sentiment = pd.read_csv('data/fear_greed_index.csv')

trades.head(), sentiment.head()


(                                      Account  Coin  Execution Price  \
 0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
 1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
 2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
 3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
 4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   
 
    Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
 0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
 1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
 2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
 3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
 4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   
 
    Closed PnL                                   Transaction Hash     Order ID  \
 0      

In [7]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00


In [9]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas


In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

try:
    from reportlab.lib.pagesizes import A4
    from reportlab.pdfgen import canvas
    pdf_enabled = True
except:
    pdf_enabled = False


base_dir = os.getcwd()
data_dir = os.path.join(base_dir, "data")
csv_dir = os.path.join(base_dir, "csv_files")
output_dir = os.path.join(base_dir, "outputs")
report_path = os.path.join(base_dir, "ds_report.pdf")

os.makedirs(csv_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


trades = pd.read_csv(os.path.join(data_dir, "historical_data.csv"))
sentiment = pd.read_csv(os.path.join(data_dir, "fear_greed_index.csv"))

print(" Loaded Trades:", trades.shape)
print(" Loaded Sentiment:", sentiment.shape)


trades.columns = trades.columns.str.lower()
sentiment.columns = sentiment.columns.str.lower()


trade_date_col = next((c for c in trades.columns if "time" in c or "date" in c), None)
sent_date_col = next((c for c in sentiment.columns if "date" in c), None)

trades["trade_date"] = pd.to_datetime(trades[trade_date_col], errors="coerce").dt.date
sentiment["sent_date"] = pd.to_datetime(sentiment[sent_date_col], errors="coerce").dt.date


sentiment_col = next(
    (c for c in sentiment.columns if "class" in c or "sent" in c or "fear" in c or "greed" in c),
    None
)

if sentiment_col is None:
    raise ValueError(" Could not find sentiment/Fear-Greed column in dataset")

sentiment = sentiment.rename(columns={sentiment_col: "market_sentiment"})


merged = trades.merge(
    sentiment[["sent_date", "market_sentiment"]],
    left_on="trade_date",
    right_on="sent_date",
    how="left"
)

merged["market_sentiment"] = merged["market_sentiment"].astype(str).str.lower()


pnl_col = next(
    (c for c in merged.columns if "pnl" in c.lower() or "profit" in c.lower()),
    None
)

if pnl_col is None:
    raise ValueError(" No PnL or profit column found in trades dataset")

merged = merged.rename(columns={pnl_col: "closedpnl"})
merged["closedpnl"] = pd.to_numeric(merged["closedpnl"], errors="coerce")


merged = merged.dropna(subset=["market_sentiment", "closedpnl"])
merged = merged[merged["market_sentiment"].isin(["fear", "greed", "neutral"])]

print(" Final merged dataset:", merged.shape)


summary = merged.groupby("market_sentiment").agg(
    num_trades=("closedpnl", "count"),
    total_pnl=("closedpnl", "sum"),
    avg_pnl=("closedpnl", "mean"),
    win_rate=("closedpnl", lambda x: (x > 0).mean())
).reset_index()

summary.to_csv(os.path.join(csv_dir, "summary_by_sentiment.csv"), index=False)


if "account" in merged.columns:
    account_stats = merged.groupby(["account", "market_sentiment"]).agg(
        num_trades=("closedpnl", "count"),
        total_pnl=("closedpnl", "sum"),
        avg_pnl=("closedpnl", "mean"),
        win_rate=("closedpnl", lambda x: (x > 0).mean())
    ).reset_index()

    account_stats.to_csv(os.path.join(csv_dir, "account_sentiment_stats.csv"), index=False)


merged.to_csv(os.path.join(csv_dir, "merged_sentiment_trades_full.csv"), index=False)


unique_sentiments = merged["market_sentiment"].nunique()

if unique_sentiments >= 2:

    sns.set(style="whitegrid")


    plt.figure(figsize=(8, 5))
    sns.boxplot(data=merged, x="market_sentiment", y="closedpnl")
    plt.title("PnL Distribution by Market Sentiment")
    plt.savefig(os.path.join(output_dir, "pnl_by_sentiment.png"))
    plt.close()


    daily = merged.groupby("trade_date").size()
    plt.figure(figsize=(12, 4))
    daily.plot(title="Trades Per Day Timeline")
    plt.savefig(os.path.join(output_dir, "trades_per_day.png"))
    plt.close()

else:
    print(" Not enough sentiment categories for visual comparison — skipping plots")


if pdf_enabled and not summary.empty:
    c = canvas.Canvas(report_path, pagesize=A4)
    width, height = A4

    c.setFont("Helvetica-Bold", 16)
    c.drawString(40, height - 50, "Trader Market Sentiment Analysis Report")

    c.setFont("Helvetica", 10)
    y = height - 100

    for _, row in summary.iterrows():
        c.drawString(
            40, y,
            f"{row['market_sentiment'].upper()} → Trades: {row['num_trades']}, "
            f"Avg PnL: {row['avg_pnl']:.2f}, Win Rate: {row['win_rate']:.2%}"
        )
        y -= 20

    c.save()
    print(" PDF report generated")

print("\n ALL DONE — PROJECT COMPLETE!")
print(f" CSVs stored in: {csv_dir}")
print(f" Charts stored in: {output_dir}")


 Loaded Trades: (211224, 16)
 Loaded Sentiment: (2644, 4)
 Final merged dataset: (27917, 19)
 PDF report generated

 ALL DONE — PROJECT COMPLETE!
 CSVs stored in: /content/drive/MyDrive/aghil/csv_files
 Charts stored in: /content/drive/MyDrive/aghil/outputs


In [19]:
!apt install gh


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gh is already the newest version (2.83.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [22]:
import os
os.chdir('/content/drive/MyDrive/aghil')


In [ ]:
!gh auth login


78? Where do you use GitHub?  [Use arrows to move, type to filter]
> GitHub.com
  Other
7

In [3]:
!gh repo create <aghilmavannoor>/aghil --public --confirm


/bin/bash: line 1: aghilmavannoor: No such file or directory
